In [ ]:
!pip install lark langchain langchain_community openai chromadb tiktoken langchain-openai

In [ ]:
from langchain.schema import Document
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma, FAISS

# Load environment variables from .env file
#load_dotenv()
openai_api_key = "your-api-key"

embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key)

<ipython-input-2-bf69677a6b9c>:9: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key)


In [3]:
import pandas as pd
df = pd.read_excel("products.xlsx")
df.head()

,id,name,price,Brand,category,keyword,weight,pack_type,pack size,food_tag,free_from
0,1,Napolina Chopped Tin Tomatoes 4 pack,2.50,Napolina,Tin Vegetables,"Tomato, chopped tomato",1.6Kg,Tin,4,"vegan ,pure_veg, egg_veg, halal","lactose, egg, Gluten, Nuts"
1,2,Heinz chopped Tomato,1.25,Heinz,Tin Vegetables,"Tomato, chopped tomato",400g,Tin,1,"vegan ,pure_veg, egg_veg, halal","lactose, egg, Gluten, Nuts"
2,3,Baxters Lentil & Bacon Soup,1.50,Baxters,Tin Soup,"Lentil, Bacon, soup",400g,Tin,1,meat,"lactose, egg, Gluten, Nuts"
3,4,Bens Egg Fried Rice,1.20,Bens,Microwave Rice,"Rice, fried rice",250g,packet,1,"egg_veg, halal","lactose, Gluten, Nuts"
4,5,Campbell's Cream of Mushroom Soup,1.50,Campbell's,Tin Soup,"Mushroom, soup",300g,Tin,1,"vegan ,pure_veg, egg_veg, halal","lactose, egg, Gluten, Nuts"


In [4]:
column_list = df.columns.tolist()
print(column_list)

['id', 'name', 'price', 'Brand', 'category', 'keyword', 'weight', 'pack_type', 'pack size', 'food_tag', 'free_from']


In [5]:
index = column_list.index("name")

# Pop the item at that index
popped_item = column_list.pop(index)
popped_item

'name'

In [6]:
# Assuming a Document class is structured as follows
from dataclasses import dataclass
from typing import List, Dict
@dataclass
class Document:
    page_content: str
    metadata: Dict

def convert_df_to_documents(df):
  documents = []
  for _, row in df.iterrows():
      # `name` becomes the page_content
      page_content = row['name']

      # The rest of the columns become metadata
      metadata = row.drop('name').to_dict()

      documents.append(Document(page_content=page_content, metadata=metadata))

  return documents
doc = convert_df_to_documents(df)

print(doc)

[Document(page_content='Napolina Chopped Tin Tomatoes 4 pack', metadata={'id': 1, 'price': 2.5, 'Brand': 'Napolina', 'category': 'Tin Vegetables', 'keyword': 'Tomato, chopped tomato', 'weight': '1.6Kg', 'pack_type': 'Tin', 'pack size': 4, 'food_tag': 'vegan ,pure_veg, egg_veg, halal', 'free_from': 'lactose, egg, Gluten, Nuts'}), Document(page_content='Heinz chopped Tomato', metadata={'id': 2, 'price': 1.25, 'Brand': 'Heinz', 'category': 'Tin Vegetables', 'keyword': 'Tomato, chopped tomato', 'weight': '400g', 'pack_type': 'Tin', 'pack size': 1, 'food_tag': 'vegan ,pure_veg, egg_veg, halal', 'free_from': 'lactose, egg, Gluten, Nuts'}), Document(page_content='Baxters Lentil & Bacon Soup', metadata={'id': 3, 'price': 1.5, 'Brand': 'Baxters', 'category': 'Tin Soup', 'keyword': 'Lentil, Bacon, soup', 'weight': '400g', 'pack_type': 'Tin', 'pack size': 1, 'food_tag': 'meat', 'free_from': 'lactose, egg, Gluten, Nuts'}), Document(page_content='Bens Egg Fried Rice', metadata={'id': 4, 'price': 1.

In [7]:
# Step 1: Data Processing
# Add as many documents as possible for the application.
# Use some functions to convert the files in the following pattern for easier querying. Make use of using good metadata names.
from langchain_community.vectorstores.utils import filter_complex_metadata

vectorstore = Chroma.from_documents(doc, embeddings)

print("Chroma vectorstore created successfully!")

Chroma vectorstore created successfully!


In [11]:
#from langchain.llms import OpenAI
from langchain_openai import OpenAI
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.chains.query_constructor.base import AttributeInfo

metadata_field_info = [
    AttributeInfo(
        name="ID",
        description="A unique identifier for each product, used for tracking and referencing the product in the database.",
        type="string",
    ),
    AttributeInfo(
        name="price",
        description="The price of the product in British pounds (GBP). This is a numeric value that indicates the cost to purchase the product.",
        type="float",
    ),
    AttributeInfo(
        name="brand",
        description="The brand name associated with the product. This could be the manufacturer's name or a commercial brand name.",
        type="string",
    ),
    AttributeInfo(
        name="category",
        description="The specific category or classification of the product, such as 'frozen vegetables', 'snacks', or 'dairy'. This helps organize products into broader groups.",
        type="string",
    ),
    AttributeInfo(
        name="keywords",
        description="A set of words or phrases that are associated with the product, typically used for search or filtering purposes. Examples include product type, main ingredients, or use case.",
        type="string",
    ),
    AttributeInfo(
        name="weight",
        description="The weight of the product, generally provided in grams (g) or kilograms (kg). This reflects the net weight of the product as it is sold.",
        type="float",
    ),
    AttributeInfo(
        name="pack_type",
        description="The type of packaging used for the product, such as 'tin', 'packet', 'box', or 'bag'. This indicates how the product is packaged for sale and storage.",
        type="string",
    ),
    AttributeInfo(
        name="pack_size",
        description="The quantity of individual items within a single package or container. For example, it could represent the number of tins or packets per pack.",
        type="int",
    ),
    AttributeInfo(
        name="food_tags",
        description="A list of dietary or food-related tags describing the product, such as 'vegan', 'pure_veg', 'egg_veg', 'halal', etc. These tags describe the product’s suitability for specific dietary preferences or restrictions.",
        type="string",
    ),
    AttributeInfo(
        name="free_from",
        description="A list of allergens or substances that the product is free from, such as 'lactose', 'gluten', 'egg', 'dairy', etc. This information is crucial for consumers with allergies or specific dietary needs.",
        type="string",
    ),
]

document_content_description = (
    "A detailed description of the product, including key information such as the product name, unique ID, "
    "price in GBP, weight in grams or kilograms, packaging type (e.g., tin, packet, box), and the number of items per package. "
    "It also includes dietary and product-related tags such as vegan, pure vegetarian, egg-based, halal, and other relevant labels, "
    "as well as a list of allergens or substances the product is free from, such as lactose, gluten, egg, or dairy."
)

llm = OpenAI(openai_api_key=openai_api_key, temperature=0)

retriever = SelfQueryRetriever.from_llm(
    llm,
    vectorstore,
    document_content_description,
    metadata_field_info,
    enable_limit=True,  # To limit the returning documents count
    verbose=True,
)

In [12]:
# Get the relevant documents
relevant_docs = retriever.get_relevant_documents("Tomatoes in tins")
print(relevant_docs)
# Iterate over the documents and print the required fields
for doc in relevant_docs:
    print(f"ID: {doc.metadata['ID']} , Name: {doc.page_content}, Price: {doc.metadata['price']}")

[]


In [13]:
# Get the relevant documents
relevant_docs = retriever.get_relevant_documents("gluten free cakes")

print(relevant_docs)
# Iterate over the documents and print the required fields
for doc in relevant_docs:
    print(f"ID: {doc.metadata['ID']} , Name: {doc.page_content}, Price: {doc.metadata['price']}")

[]


## Furthur Improvements


1. Fix output bug
2. Filtering Results
3. Limit the results


## Advanced Improvements
1. Add Voice to Text Search
2. Add language Detection
3. Add translation
4. Add advanced quries for better results.
